# Noodling with ideas from On Numbers and Games

The notation introduced in On Numbers, brief demo for my future usage:

In [1]:
(require '[clojupyter.misc.display :as display])
(display/markdown "\\begin{equation}
\\{ \\emptyset | \\emptyset \\}
\\end{equation}")

\begin{equation}
\{ \emptyset | \emptyset \}
\end{equation}

Need to come up with a manageable way not to have to do all escape characters all the time.

\begin{equation}
\{\emptyset|\emptyset\}
\end{equation}

In [2]:
(defn simple-conway-notation [n1 n2] (str "\\begin{equation} \\{" n1 "|" n2 "\\}\\end{equation}"))

#'user/simple-conway-notation

In [3]:
(display/markdown (simple-conway-notation "\\emptyset" "\\emptyset"))

\begin{equation} \{\emptyset|\emptyset\}\end{equation}

In [4]:
(display/markdown (simple-conway-notation 0 1))

\begin{equation} \{0|1\}\end{equation}

## Is it a number?
Conway says that if an expression contains only numbers on the left and the right and if there is no number on the left equal to or greater than any number on the left, then the resulting expression is considered equal to a number.  Let's assume it's ok for now to represent the notation from the book as two Clojure vectors.

In [5]:
(defn is-number [left right]
    (if (= 0 (count right))
           true
           (let [least-r (reduce min right)
                 greatest-l (reduce max left)]
               (> least-r greatest-l))))

#'user/is-number

In [6]:
(is-number [1 2 3] [4 5 6])

true

In [7]:
(is-number [1 2 3] [])

true

In [8]:
(is-number [2] [1])

false

A simple case of two nonempty sets that fit the rule, then an empty set for the right that can only pass, then an example of something that is not a number.  A minor improvement must be made to the notation code to display some of these scenarios:

In [9]:
(defn simple-conway-notation [n1 n2]
    (str "\\begin{equation} \\{"
         (clojure.string/join "," n1)
         "|"
         (clojure.string/join "," n2)
         "\\}\\end{equation}"))

#'user/simple-conway-notation

In [10]:
(defn ask-is-num [left right]
    (str (simple-conway-notation left right) " "
         (if (is-number left right) "is" "is not")
         " a number"))

#'user/ask-is-num

In [11]:
(display/markdown (str
                   (ask-is-num [1 2 3] [4 5 6])
                   (ask-is-num [1 2 3] [])
                   (ask-is-num [2] [1])))

\begin{equation} \{1,2,3|4,5,6\}\end{equation} is a number\begin{equation} \{1,2,3|\}\end{equation} is a number\begin{equation} \{2|1\}\end{equation} is not a number

## Constructing some numbers in the new notation.
Taking some numbers from the constructions for basic integers in the book and fully expanding them as lists of lists proves somewhat interesting.

In [12]:
(def empty-set [])
(def zero [empty-set :bar empty-set])
(def one [[zero] :bar empty-set])
(def two [[one] :bar empty-set])
(def neg-one [empty-set :bar [zero]])

#'user/neg-one

In [13]:
two

[[[[[[] :bar []]] :bar []]] :bar []]

In [14]:
zero

[[] :bar []]

In [15]:
neg-one

[[] :bar [[[] :bar []]]]

In [16]:
(defn incr [exp] [exp :bar []])

#'user/incr

In [17]:
(incr two)

[[[[[[[] :bar []]] :bar []]] :bar []] :bar []]

In [18]:
(incr (incr two))

[[[[[[[[] :bar []]] :bar []]] :bar []] :bar []] :bar []]

In [19]:
(defn decr [exp] [[] :bar exp])

#'user/decr

In [20]:
(decr neg-one)

[[] :bar [[] :bar [[[] :bar []]]]]

In [21]:
(decr (decr neg-one))

[[] :bar [[] :bar [[] :bar [[[] :bar []]]]]]

The integers are symmetrical when you look at their negatives.  This kind of makes sense.  I thought it would be interesting to see the fully expanded set notation itself:

In [22]:
(defn simple-conway-notation [expr]
    (cond
        (= 0 (count expr))
        "\\emptyset"
        (some #(= % :bar) expr)
        (let [pre-bar (into [] (take-while #(not (= % :bar)) expr))
              post-bar (drop (+ 1 (count pre-bar)) expr)]
            (str
             "\\{"
             (clojure.string/join "," (map simple-conway-notation pre-bar))
             "|"
             (clojure.string/join "," (map simple-conway-notation post-bar))
             "\\}"))
        :else
        (clojure.string/join "," (map simple-conway-notation expr))))

#'user/simple-conway-notation

In [23]:
(display/markdown (str "\\begin{equation} -2 =" (simple-conway-notation (decr neg-one)) "\\end{equation}"))

\begin{equation} -2 =\{\emptyset|\{\emptyset|\{\emptyset|\emptyset\}\}\}\end{equation}

May as well make things easier

In [29]:
(defn print-eq [note expr] 
    (display/markdown (str "\\begin{equation}"
                           note " "
                           (simple-conway-notation expr)
                           "\\end{equation}")))

#'user/print-eq

In [31]:
(print-eq "2 =" two)

\begin{equation}2 = \{\{\{\emptyset|\emptyset\}|\emptyset\}|\emptyset\}\end{equation}